In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import pandas as pd

In [ ]:
service = Service("~/Downloads/chromedriver")
driver = webdriver.Chrome(service=service)
driver.get('https://doc.sd.gov/adult/lookup/') 

In [ ]:
sd = pd.DataFrame(columns=[])

In [ ]:
failed_ids =[]
i = 2
while True:
    xpath = f"//a[@href=\"javascript:__doPostBack('grvList','Page${i}')\"]"
    try:
        links  = driver.find_elements("xpath", "//*[contains(@id,'grvList_hypDocId_')]")
        for x in range(len(links)):
            links = driver.find_elements("xpath", "//*[contains(@id,'grvList_hypDocId_')]")
            doc = links[x].text
            links[x].click()
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'lxml')
            status = soup.find('span', {'id':'lblCorrStat'}).get_text()
            if status == 'Active Inmate':
                name = soup.find('span', {'id':'lblMainName'}).get_text()
                age = soup.find('span', {'id':'lblAge'}).get_text()
                id = soup.find('span', {'id':'lblDocId'}).get_text()
                race = soup.find('span', {'id':'lblRace'}).get_text()
                fac = soup.find('span', {'id':'lblCurrLoc'}).get_text()
                sendate_elements = soup.find_all('td', {'class': 'sentdate'})
                county_elements = soup.find_all('td', {'class': 'sentco'})
                offense_elements = soup.find_all('td', {'class': 'sentcrime'})
                sentence_elements = soup.find_all('td', {'class': 'sentlen'})

                sendates = []
                for sendate in sendate_elements:
                    sendates.append(sendate.get_text())

                offenses = []
                for offense in offense_elements:
                    offenses.append(offense.get_text())

                counties = []
                for county in county_elements:
                    counties.append(county.get_text())

                sentences = []
                for sentence in sentence_elements:
                    sentences.append(sentence.get_text())
                
                crimes = [list(pair) for pair in zip(offenses, sentences, sendates, counties)]
                rows = []
                for pair in crimes:
                    row = {
                        'ID': id,
                        'Name': name,
                        'Race': race,
                        'Facility': fac,
                        'Offense Description': pair[0],
                        'Sentence Length': pair[1],
                        'Sentence Date': pair[2],
                        'County': pair[3]
                    }
                    rows.append(row)
                    
                sd = pd.concat([sd, pd.DataFrame(rows)])
            driver.back()
        driver.find_element('xpath', xpath).click()
        i += 1
    except Exception as e:
        print(f"Failed to scrape {doc}. Error: {e}")
        failed_ids.append(doc)
        try:
            driver.find_element('xpath', "//a[contains(text(), '< Back to Search Results')]").click()
        except:
            driver.back()
        continue


In [ ]:
sd.to_csv('/Users/tumendemberelshalkhaan/Desktop/South_Dakota_Raw.csv', index=False)